Similar Image Search Using Deep Learning on Caltech-101
=======================================================

In this notebook, we will walk you through adapting a neural network trained on the ImageNet Challenge to find similar images within the Caltech-101 dataset. At the end of the notebook, we will be able to alogithmically identify images that are visually similar to each other within the Caltech-101 dataset.

The notebook has several parts:

* Part I focuses on loading the data.

* Part II focuses on using a pre-trained neural net to extract visual features. 

* Part III focuses on using the extracted visual features to train a nearest neighbors model. 

Part I: The Data
=========================================

In this notebook, we use the Caltech-101 dataset. Caltech-101 contains photos of objects belonging to 101 categories. **Note: This is a large dataset, so it may take a while to download.**It was collected by Fei-Fei Li, Marco Andreetto, and Marc 'Aurelio Ranzato in September 2003.


In [1]:
import graphlab 
images = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images')

2016-04-27 10:21:30,622 [WARNING] graphlab.product_key, 191: Unable to write current GraphLab Create license to /home/prathamesh/.graphlab/config. Ensure that this user account                         has write permission to /home/prathamesh/.graphlab/config to save the license for offline use.
2016-04-27 10:21:32,123 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1461732681.log


This non-commercial license of GraphLab Create is assigned to prathamesh.tugaonkar@vit.edu.in and will expire on April 09, 2017. For commercial licensing options, visit https://dato.com/buy/.


Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/dir_archive.ini to /var/tmp/graphlab-prathamesh/3455/af080781-132d-448b-9834-a5a54a606ec8.ini

Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/objects.bin to /var/tmp/graphlab-prathamesh/3455/2ee4c2cd-3b25-4e12-a3c0-b3fec9e90d8c.bin

Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/m_96330e5f81c95d09.frame_idx to /var/tmp/graphlab-prathamesh/3455/c4128949-b964-42d5-9fa8-d95b72d3bc70.frame_idx

Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/m_96330e5f81c95d09.sidx to /var/tmp/graphlab-prathamesh/3455/6df95dd8-01df-413f-b5d6-9de906ff6335.sidx

Part II: Extracting Features 
=========================================

We use the neural network trained on the 1.2 million images of the ImageNet Challenge. For each image of the Caltech-101 dataset, we take the activations of the layer before the classification layer and consider that our feature vector for the image. This is a sort of internal representation of what the network knows about the image. If these feature vectors are similar for two images, then the images should be similar as well. This concept is covered more thoroughly in our [blog post](http://blog.graphlab.com/deep-learning-blog-post) on the subject. Note that feature exctraction will not be feasible without a GPU and the [GPU installation](http://graphlab.com/products/create/gpu_install.html). **In that case, you should download the SArray that contains the result of this step.** 

In [ ]:
# Only do this if you have a GPU
#pretrained_model = graphlab.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
#images['extracted_features'] = pretrained_model.extract_features(images)

# If you do not have a GPU, do this instead. 
images['extracted_features'] = graphlab.SArray('http://s3.amazonaws.com/dato-datasets/deeplearning/pre_extracted_features.gl')


**Now, let's inspect the images SFrame. The 'extracted_features' column contains  vector representations of the data, as we expected it to. **

In [8]:
images

Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/m_96330e5f81c95d09.0000 to /var/tmp/graphlab-prathamesh/3455/5f38ee4c-c668-47a2-bd95-df5154981c05.0000

Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/m_96330e5f81c95d09.0001 to /var/tmp/graphlab-prathamesh/3455/fdae7b39-f406-4239-a79a-9551dff6b757.0001

Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/m_96330e5f81c95d09.0002 to /var/tmp/graphlab-prathamesh/3455/9e3ffab6-f7f7-41ca-9f9d-20f7397d1d6c.0002

Downloading http://s3.amazonaws.com/dato-datasets/caltech_101/caltech_101_images/m_96330e5f81c95d09.0003 to /var/tmp/graphlab-prathamesh/3455/767ccac6-c6d5-4875-addc-32ca3ce35a62.0003

Downloading http://s3.amazonaws.com/dato-datasets/deeplearning/pre_extracted_features.gl/m_3e736cfeecc9a07f.0000 to /var/tmp/graphlab-prathamesh/3455/4dafb89c-1594-470b-871e-981f12ea4cab.0000

Downloading http://s3.amazonaws.com/dato-datasets/deeplearning/pre_extracted_features.gl/m_3e736cfeecc9a07f.0001 to /var/tmp/graphlab-prathamesh/3455/6ecb8aab-7177-4bd1-bb6f-53cfab0f415c.0001

Downloading http://s3.amazonaws.com/dato-datasets/deeplearning/pre_extracted_features.gl/m_3e736cfeecc9a07f.0002 to /var/tmp/graphlab-prathamesh/3455/a7f05054-e924-48b1-aaf6-d8cef89199f5.0002

Downloading http://s3.amazonaws.com/dato-datasets/deeplearning/pre_extracted_features.gl/m_3e736cfeecc9a07f.0003 to /var/tmp/graphlab-prathamesh/3455/16f55a46-3ea0-4c91-92a4-427f6557860d.0003

Columns:
	image	Image
	extracted_features	array

Rows: 9144

Data:
+------------------------+-------------------------------+
|         image          |       extracted_features      |
+------------------------+-------------------------------+
| Height: 256 Width: 256 | [0.0, 1.23551917076, 0.780... |
| Height: 256 Width: 256 | [0.0, 0.0, 0.0, 0.0, 2.215... |
| Height: 256 Width: 256 | [0.700446844101, 0.0, 0.0,... |
| Height: 256 Width: 256 | [0.0, 2.67075538635, 1.840... |
| Height: 256 Width: 256 | [1.50202608109, 0.42167723... |
| Height: 256 Width: 256 | [0.0, 0.0, 0.0, 0.0, 0.103... |
| Height: 256 Width: 256 | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| Height: 256 Width: 256 | [0.0, 0.0, 0.0, 0.93841451... |
| Height: 256 Width: 256 | [2.32718586922, 1.89031362... |
| Height: 256 Width: 256 | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
+------------------------+-------------------------------+
[9144 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

Part III: Finding similar images via Nearest Neighbors on Extracted Features
=========================================

Knowing that similar extracted features should mean visually similar images, we can do a similar image search simply by finding an images nearest neighbors in the feature space. We demonstrate this below. 

**First, we construct the nearest neighbors model on the extracted features. This will allow us to see each image's closest neighbor**

In [9]:
nearest_neighbor_model = graphlab.nearest_neighbors.create(images, features=['extracted_features'])

Starting brute force nearest neighbors model training.

In [10]:
similar_images = nearest_neighbor_model.query(images, k = 2)

Starting blockwise querying.

max rows per data block: 7668

number of reference data blocks: 4

number of query data blocks: 2

+--------------+---------+-------------+--------------+

...

| 4572         | 3.1e+07 | 37.5        | 25.95s       |

| 9144         | 5.2e+07 | 62.5        | 50.77s       |

| 9144         | 7.3e+07 | 87.5        | 51.94s       |

| Done         | 8.4e+07 | 100         | 51.99s       |

+--------------+---------+-------------+--------------+

**similar_images is an SFrame which contains a query label, and it's neighbor, the reference label**

In [11]:
similar_images

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 18288

Data:
+-------------+-----------------+------------------+------+
| query_label | reference_label |     distance     | rank |
+-------------+-----------------+------------------+------+
|      0      |        0        |       0.0        |  1   |
|      0      |       1535      |  30.6212798551   |  2   |
|      1      |        1        | 2.6973983047e-06 |  1   |
|      1      |       8990      |  54.1374904338   |  2   |
|      2      |        2        |       0.0        |  1   |
|      2      |       8339      |  36.5163977404   |  2   |
|      3      |        3        |       0.0        |  1   |
|      3      |       3633      |  28.8060467874   |  2   |
|      4      |        4        |       0.0        |  1   |
|      4      |       140       |  30.5404123745   |  2   |
+-------------+-----------------+------------------+------+
[18288 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

**We do some cleaning to remove the instances where the query equals the reference. This happened beacause the query set was identical to the reference set**

In [12]:
similar_images = similar_images[similar_images['query_label'] != similar_images['reference_label']]

In [13]:
similar_images

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: Unknown

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |       1535      | 30.6212798551 |  2   |
|      1      |       8990      | 54.1374904338 |  2   |
|      2      |       8339      | 36.5163977404 |  2   |
|      3      |       3633      | 28.8060467874 |  2   |
|      4      |       140       | 30.5404123745 |  2   |
|      5      |       6206      | 46.2503776144 |  2   |
|      6      |       2331      | 22.8088822589 |  2   |
|      7      |       6466      | 47.3209057413 |  2   |
|      8      |       5760      | 22.8112886214 |  2   |
|      9      |       1710      | 61.5027370061 |  2   |
+-------------+-----------------+---------------+------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use len(sf) to force materialization.

**Now we can explore similar images. For instance, the closest image to image 9 is image 1710. We can view and see both are starfish**

In [15]:
graphlab.canvas.set_target('ipynb')
graphlab.SArray([images['image'][9]]).show()

In [16]:
graphlab.SArray([images['image'][1710]]).show()

**Similarly, images 0 and 1535 are two similar photos of the same person**

In [17]:
graphlab.SArray([images['image'][0]]).show()

In [18]:
graphlab.SArray([images['image'][1535]]).show()